In [1]:
import pandas as pd
from utils import load_config, MongoDBSession, build_user_history

In [2]:
# parameter
query_cfg_path = './config/query.yaml'
rec_result_file = 'xindong_head-202301052258.csv'
topK = 10

In [3]:
# load query config
query_config = load_config(query_cfg_path)

In [4]:
# init mongoDB
mongodb_session = MongoDBSession(query_config['connect_settings'])
database_names = mongodb_session.database_names
database_names

['xindong123', 'xindong_inference', 'xindong']

In [5]:
# get xindong.media(my)(id, title, tags)
mongodb_session.init_db(db_name='xindong')
media_cl = mongodb_session.query(collect_name='media')

ready db(=xindong)....


In [16]:
# build item2info
item2info = dict()
for record in media_cl:
    item2info[record['ITEM_ID']] = {
                        'title' : record['title'],
                        'tags' : record['tags-data']
                        }

In [6]:
# get All/user2meta_history data
mongodb_session.init_db(db_name='xindong_inference')
user2meta_history_cl = mongodb_session.query(collect_name='All/user2meta_history')
user2history_cl = mongodb_session.query(collect_name='All/user2history')

ready db(=xindong_inference)....


In [8]:
user2meta_history_cl = build_user_history(user2meta_history_cl, user2history_cl)

In [9]:
# build user2meta_history
num_list = []
user2meta_history = dict()
for record in user2meta_history_cl:
    num_list.append(len(record['history']))
    user2meta_history[record['USER_ID']] = record['history']

In [14]:
# get recommend results
dat = pd.read_csv('csv_file/' + rec_result_file)
recommend_results = list(dat.to_dict('records'))
dat.head(2)

,userId,rank_itemId
0,342121,"[87728, 21086, 21351, 28840, 21575, 25707, 311..."
1,342122,"[21261, 23813, 83242, 19912, 20023, 22183, 236..."


In [11]:
# get item2item_rank
mongodb_session.init_db(db_name='xindong_inference')
item2item_rank_cl = mongodb_session.query(collect_name='All/item2item_rank')
recommend_results = item2item_rank_cl[:]

ready db(=xindong_inference)....


In [17]:
# main
history_user = list()
for record in recommend_results:
    meta_history = user2meta_history[record['userId']]
    print('user_id : ', record['userId'])
    # meta item
    for item_id in meta_history:
        print('meta_item : ', item_id)
        print('meta_item(title) : ', item2info[item_id]['title'])
        print('meta_item(tags) : ', item2info[item_id]['tags'])
        print('-----------------------')
    print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
    # rec item
    for item_id in eval(record['rank_itemId'])[:topK]:
        print('rec_item : ', item_id)
        print('rec_item(title) : ', item2info[item_id]['title'])
        print('rec_item(tags) : ', item2info[item_id]['tags'])
        print('-----------------------')  
    history_user.append(record['userId'])
    input('====================================')

user_id :  342121
meta_item :  28475
meta_item(title) :  疫情期间开发的情人
meta_item(tags) :  [None]
-----------------------
meta_item :  20798
meta_item(title) :  找来极品身材美女按在沙发干
meta_item(tags) :  [{'_id': 13383, 'name': '清纯妹子'}, {'_id': 17790, 'name': '91大神'}, {'_id': 17782, 'name': '探花'}]
-----------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
rec_item :  87728
rec_item(title) :  某网新签约36F高颜值极品身材性感美女和富二代男友各种场景啪啪艳舞诱惑流出颜值一流 -2
rec_item(tags) :  []
-----------------------
rec_item :  21086
rec_item(title) :  极品身材美女被插得抽畜了
rec_item(tags) :  [{'_id': 17782, 'name': '探花'}]
-----------------------
rec_item :  21351
rec_item(title) :  停车场上演超跑情人淫梦
rec_item(tags) :  [{'_id': 17754, 'name': '玩具'}, {'_id': 17746, 'name': '车震'}, None]
-----------------------
rec_item :  28840
rec_item(title) :  我的情人！口活一流
rec_item(tags) :  [{'_id': 13331, 'name': '少妇'}, {'_id': 13336, 'name': '深喉'}]
-----------------------
rec_item :  21575
rec_item(title) :  美少女【粉色情人】极品水蛇腰
rec_item(tags) :  [{'_id': 13338, 'name': '萝莉'

In [11]:
record.keys()

dict_keys(['_id', 'ITEM_ID', 'ITEM_ID_RANK'])

In [26]:
eval(record['rank_itemId'])

[90072,
 96417,
 96472,
 19881,
 61874,
 96329,
 20182,
 91387,
 33871,
 21463,
 96374,
 30027,
 20930,
 32682,
 29341,
 96598,
 95052,
 32494,
 96578,
 96331,
 96352,
 48527,
 73689,
 57216,
 96599,
 30827,
 30145,
 32986,
 96466,
 73748,
 83731,
 96318,
 21909,
 28775,
 96409,
 96317,
 95063,
 61998,
 74706,
 96263,
 96383,
 96440,
 54707,
 29054,
 91920,
 26002,
 20144,
 30139,
 74102,
 96283,
 96437,
 96475,
 19713,
 47045,
 28684,
 75430,
 96448,
 94600,
 96520,
 97687,
 87723,
 75522,
 64741,
 94272,
 21465,
 96193,
 60234,
 71833,
 74704,
 96361,
 96365,
 72622,
 72183,
 19835,
 21452,
 56513,
 20986,
 58994,
 22085,
 96103,
 29423,
 93119,
 96393,
 39140,
 94291,
 97541,
 87492,
 74899,
 32519,
 30847,
 26316,
 59244,
 74784,
 28738,
 96219,
 19987,
 97529,
 96588,
 33882,
 32527,
 92279,
 19772,
 72805,
 81939,
 21471,
 73882,
 76230,
 21621,
 83787,
 20802,
 96494,
 96604,
 96267,
 74349,
 21944,
 21803,
 89574,
 89729,
 25490,
 21876,
 96350,
 29157,
 96411,
 29385,
 39303,
